<a href="https://colab.research.google.com/github/Ashrithajangam/IT7133-FALL24/blob/main/Assignment_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Sentiment Analysis - Classification

Quite often, the sentiment analysis can be treated as classification. In this demostration, we will use the google play dataset, available at
https://www.kaggle.com/datasets/prakharrathi25/google-play-store-reviews

First, install libraries and mount the drive

In [ ]:
!pip install transformers datasets evaluate

In [ ]:
# Step 1: Mount Google Drive to load the dataset and save the model
from google.colab import drive
drive.mount('/content/drive')

# Step 2: Import necessary libraries
import os
import pandas as pd

# Step 3: Verify if the file exists in the directory
print("Files in MyDrive:")
print(os.listdir('/content/drive/MyDrive'))

# Step 4: Load the dataset (replace with your actual file path if different)
data = pd.read_csv('/content/drive/MyDrive/Data/IMDB Dataset.csv', encoding="ISO-8859-1")

# Sample 5000 rows from the dataset for training
data_sample = data.sample(n=5000, random_state=42)

# Preprocess the data by mapping labels (positive/negative) to numerical values
label2id = {'positive': 1, 'negative': 0}
id2label = {1: 'positive', 0: 'negative'}

# Rename columns to 'text' and 'label'
data_sample = data_sample[['review', 'sentiment']]
data_sample.columns = ['text', 'label']
data_sample["label"] = data_sample["label"].map(label2id)

# Display the preprocessed data
print(data_sample.head())


# Step 8: Prepare the data collator
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer, return_tensors="tf")

# Step 9: Set up the optimizer and model for training
import evaluate
import numpy as np
import tensorflow as tf
from transformers import create_optimizer
from transformers import TFAutoModelForSequenceClassification
from transformers.keras_callbacks import KerasMetricCallback

# Load accuracy evaluation metric
accuracy = evaluate.load("accuracy")

# Define compute_metrics function for accuracy
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return accuracy.compute(predictions=predictions, references=labels)

# Set training parameters
batch_size = 16
num_epochs = 3
batches_per_epoch = len(tokenized_data["train"]) // batch_size
total_train_steps = int(batches_per_epoch * num_epochs)
optimizer, schedule = create_optimizer(init_lr=2e-5, num_warmup_steps=0, num_train_steps=total_train_steps)

# Load DistilBERT model for sequence classification
model = TFAutoModelForSequenceClassification.from_pretrained(
    "distilbert-base-uncased", num_labels=2, id2label=id2label, label2id=label2id
)

# Step 10: Prepare the TensorFlow datasets for training and validation
tf_train_set = model.prepare_tf_dataset(
    tokenized_data["train"],
    shuffle=True,
    batch_size=batch_size,
    collate_fn=data_collator,
)

tf_validation_set = model.prepare_tf_dataset(
    tokenized_data["test"],
    shuffle=False,
    batch_size=batch_size,
    collate_fn=data_collator,
)

# Step 11: Compile the model and set up training
model.compile(optimizer=optimizer)
metric_callback = KerasMetricCallback(metric_fn=compute_metrics, eval_dataset=tf_validation_set)

# Step 12: Train the model
model.fit(x=tf_train_set, validation_data=tf_validation_set, epochs=num_epochs, callbacks=[metric_callback])

# Step 13: Save the trained model to Google Drive
model.save_pretrained("/content/drive/MyDrive/Model/sentiment_classification_model")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Files in MyDrive:
['WIDER_train', 'Snort Running and Texting', 'Colab Notebooks', 'Datasets', 'Model', 'Old', 'predicted_charges.csv', 'IMDB Dataset.csv', 'Data', 'IMDB Dataset.gsheet']
                                                    text  label
33553  I really liked this Summerslam due to the look...      1
9427   Not many television shows appeal to quite as m...      1
199    The film quickly gets to a major chase scene w...      0
12447  Jane Austen would definitely approve of this o...      1
39489  Expectations were somewhat high for me when I ...      0


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

...

Map:   0%|          | 0/4000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFDistilBertForSequenceClassification: ['vocab_layer_norm.bias', 'vocab_transform.bias', 'vocab_transform.weight', 'vocab_layer_norm.weight', 'vocab_projector.bias']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFDistilBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['pre_classifier.weight', 'pre_classifier.bias', 'classifier.weight', 'classifier.bias']
You should 

Epoch 1/3
250/250 [==============================] - 315s 1s/step - loss: 0.3885 - val_loss: 0.2430 - accuracy: 0.8990
Epoch 2/3
250/250 [==============================] - 273s 1s/step - loss: 0.1750 - val_loss: 0.2346 - accuracy: 0.9130
Epoch 3/3
250/250 [==============================] - 271s 1s/step - loss: 0.0945 - val_loss: 0.2555 - accuracy: 0.9120
Predicted sentiment: positive


In [ ]:
text = "I'm good!!!"

In [ ]:
inputs = tokenizer(text, return_tensors="tf")

In [ ]:
# from transformers import TFAutoModelForSequenceClassification

# model = TFAutoModelForSequenceClassification.from_pretrained("stevhliu/my_awesome_model")
logits = model(**inputs).logits

In [ ]:
predicted_class_id = int(tf.math.argmax(logits, axis=-1)[0])
model.config.id2label[predicted_class_id]

'positive'